# CSE258 Assignment 1

### Rating

In [1]:
import gzip
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from collections import defaultdict

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def readJson(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return df

df = readJson('assignment1/train.json.gz')

In [3]:
Data_total = [df[x] for x in range(0,len(df))]
Data_train = [df[x] for x in range(0,len(df)/2)]
Data_valid = [df[x] for x in range(len(df)/2,len(df))]

In [4]:
df2 = readJson('assignment1/test_Helpful.json.gz')
Data_test = [df2[x] for x in range(0,len(df2))]

### Rating

In [5]:
# allRatings = []
# userRatings = defaultdict(list)
# itemRatings = defaultdict(list)
# for l in Data_total:
#     user,item = l['reviewerID'],l['itemID']
#     allRatings.append(l['rating'])
#     userRatings[user].append(l['rating'])
#     itemRatings[item].append(l['rating'])

# globalAverage = sum(allRatings) / len(allRatings)
# userAverage = {}
# itemAverage = {}
# for u in userRatings:
#     userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
# for i in itemRatings:
#     itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

# RatingPred_valid = []
# for l in Data_valid:
#     u,i = l['reviewerID'], l['itemID']
#     if u in userAverage and i in itemAverage:
#         RatingPred_valid.append((globalAverage + userAverage[u] + itemAverage[i]) / 3)
#     elif u in userAverage:
#         RatingPred_valid.append((globalAverage + userAverage[u]) / 2)
#     elif i in itemAverage:
#         RatingPred_valid.append((globalAverage + itemAverage[i]) / 2)
#     else:
#         RatingPred_valid.append(globalAverage)

# Rating_valid = [x['rating'] for x in Data_valid]
# MSE = mean_squared_error(RatingPred_valid, Rating_valid)
# MSE

# predictions = open("assignment1/predictions_Rating.txt", 'w')
# for l in open("assignment1/pairs_Rating.txt"):
#     if l.startswith("userID"):
#         #header
#         predictions.write(l)
#         continue
#     u,i = l.strip().split('-')
#     if u in userAverage and i in itemAverage:
#         predictions.write(u + '-' + i + ',' + str((userAverage[u] + itemAverage[i]) / 2) + '\n')
#     elif u in userAverage:
#         predictions.write(u + '-' + i + ',' + str(userAverage[u]) + '\n')
#     elif i in itemAverage:
#         predictions.write(u + '-' + i + ',' + str(itemAverage[i]) + '\n')
#     else:
#         predictions.write(u + '-' + i + ',' + str(globalAverage) + '\n')

# predictions.close()

In [35]:
rating_valid = [x['rating'] for x in Data_valid]
Data_train = Data_total
rating_train = [x['rating'] for x in Data_train]

In [95]:
def calLoss(a, beta_u, beta_i, lmd):
    pred_train = []
    for x in Data_train:
        r = a
        if (x['reviewerID'] in I_u):
            r += 1.3 * beta_u[x['reviewerID']]
        if (x['itemID'] in U_i):
            r += 0.87 * beta_i[x['itemID']]
        pred_train.append(r)
    MSE = mean_squared_error(pred_train, rating_train) + lmd * (sum([beta_u[x] ** 2 for x in beta_u]) / len(beta_u) + sum([beta_i[x] ** 2 for x in beta_i]) / len(beta_i))
    return MSE

In [96]:
I_u = defaultdict(int) # number of items each user bought
U_i = defaultdict(int) # number of buyers for each item
for x in Data_train:
    I_u[x['reviewerID']] += 1
    U_i[x['itemID']] += 1

def regression (lmd):        
#     initialize
    a = 3
    beta_u = {}
    for x in I_u:
        beta_u[x] = np.random.random()
    beta_i = {}
    for x in U_i:
        beta_i[x] = np.random.random()

    count = 0
    MSE_list = []
    a_list = []
    beta_u_list = []
    beta_i_list = []
    while count < 5:
        a = sum([x['rating'] - 1.13 * beta_u[x['reviewerID']] - 0.87 * beta_i[x['itemID']] for x in Data_train]) / len(Data_train)
        a_list.append(a)
        MSE = calLoss(a,beta_u,beta_i,lmd)
        MSE_list.append(MSE)
        if len(MSE_list) > 1:
            if MSE >= MSE_list[-2]:
                count += 1
            else:
                count = 0
        
        for x in I_u:
            beta_u[x] = 0.0
        for x in Data_train: 
            beta_u[x['reviewerID']] += (x['rating'] - a - 0.87 * beta_i[x['itemID']]) / (lmd + I_u[x['reviewerID']])
        beta_u_list.append(beta_u)
        MSE = calLoss(a,beta_u,beta_i,lmd)
        MSE_list.append(MSE)
        if len(MSE_list) > 1:
            if MSE >= MSE_list[-2]:
                count += 1
            else:
                count = 0
                
        for x in U_i:
            beta_i[x] = 0.0
        for x in Data_train:
            beta_i[x['itemID']] += (x['rating'] - a - 1.13 * beta_u[x['reviewerID']]) / (lmd + U_i[x['itemID']])
        beta_i_list.append(beta_i)
        MSE = calLoss(a,beta_u,beta_i,lmd)
        MSE_list.append(MSE)
        if len(MSE_list) > 1:
            if MSE >= MSE_list[-2]:
                count += 1
            else:
                count = 0
#         pred_valid = []
#         for x in Data_valid:
#             r = a
#             if (x['reviewerID'] in I_u):
#                 r += beta_u[x['reviewerID']]
#             if (x['itemID'] in U_i):
#                 r += beta_i[x['itemID']]
#             pred_valid.append(r)
#         MSE = mean_squared_error(pred_valid, rating_valid)
#         MSE_list.append(MSE)
#         if len(MSE_list) > 1:
#             if MSE >= MSE_list[-2]:
#                 count += 1
#             else:
#                 count = 0
                
    return MSE_list, a_list, beta_u_list, beta_i_list

In [97]:
MSE_list, a_list, beta_u_list, beta_i_list = regression(5.6)

In [98]:
MSE_list

[5.1304927419484363,
 3.7862391477422723,
 2.9599885150011911,
 2.8861791142505537,
 2.5894701591447418,
 2.33117019970364,
 2.2846417367593994,
 2.2024043733382426,
 1.9807982021525592,
 1.9550322019467181,
 1.9275355237193512,
 1.7888842385877091,
 1.7758228501777529,
 1.766303895052777,
 1.6911371446874366,
 1.6849055225545773,
 1.6815059612484509,
 1.6440632204558545,
 1.6412175886931317,
 1.6399538384369341,
 1.6223328632072742,
 1.6210757438533905,
 1.6205800435982005,
 1.6126220453104478,
 1.6120808749607591,
 1.611872659731147,
 1.6083903115836522,
 1.608162136574125,
 1.6080674840858451,
 1.6065818465693078,
 1.6064872658098102,
 1.6064406500048594,
 1.6058203668727851,
 1.6057817156064238,
 1.6057570794243525,
 1.6055031364536778,
 1.6054875302004987,
 1.6054737791446119,
 1.6053718471413023,
 1.6053656104294696,
 1.6053576373439076,
 1.6053176343751614,
 1.6053151641547432,
 1.6053104267290428,
 1.6052951896904146,
 1.6052942188928923,
 1.6052913621094511,
 1.605285820009706

In [104]:
len(MSE_list)

213

In [103]:
predictions = open("assignment1/predictions_Rating.txt", 'w')
for l in open("assignment1/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    r = a_list[-6]
    if u in beta_u_list[-6]:
        r += 1.13 * beta_u_list[-6][u]
    if i in beta_i_list[-6]:
        r += 0.87 * beta_i_list[-6][i]
    if r > 5.:
        r = 5.
    if r < 1:
        r = 1.0
    predictions.write(u + '-' + i + ',' + str(r) + '\n')

predictions.close()

In [25]:
len([x for x in Data_total if x['rating'] == 0.0])

0